# LSTM-1 sin tiempo

In [8]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\carlo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\carlo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [23]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# 1. Preparación de Datos:

# Cargar el conjunto de datos
data = pd.read_csv('raw_sales.csv')

# Convertir la columna datesold a un número ordinal
data['datesold'] = pd.to_datetime(data['datesold']).apply(lambda x: x.toordinal())

# Codificar la columna propertyType
encoder = OneHotEncoder()
propertyType_encoded = encoder.fit_transform(data[['propertyType']])
feature_names = encoder.get_feature_names_out(['propertyType'])
data = pd.concat([data, pd.DataFrame(propertyType_encoded.toarray(), columns=feature_names)], axis=1)
data.drop('propertyType', axis=1, inplace=True)

# Normalizar los valores
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Generar secuencias
def generate_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length, :-1])
        y.append(data[i+sequence_length, -1])
    return np.array(X), np.array(y)

X, y = generate_sequences(data_scaled, sequence_length)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.22, random_state=42)  # 0.22 x 0.9 = 0.2

# Convertir los datos a tensores de PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# 2. Construcción del Modelo:

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.linear(out[:, -1, :])
        return out

input_dim = X_train_tensor.shape[2]
hidden_dim = 50
model = LSTMModel(input_dim, hidden_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 3. Entrenamiento y Validación:

num_epochs = 50
for epoch in range(num_epochs):
    # Entrenamiento
    optimizer.zero_grad()
    outputs_train = model(X_train_tensor)
    loss_train = criterion(outputs_train, y_train_tensor.unsqueeze(1))
    loss_train.backward()
    optimizer.step()
    
    # Validación
    model.eval()  # Poner el modelo en modo de evaluación
    with torch.no_grad():
        outputs_val = model(X_val_tensor)
        loss_val = criterion(outputs_val, y_val_tensor.unsqueeze(1))
    model.train()  # Regresar el modelo al modo de entrenamiento

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss_train.item():.4f}, Validation Loss: {loss_val.item():.4f}")

# 4. Evaluación del Conjunto de Test:

model.eval()
with torch.no_grad():
    outputs_test = model(X_test_tensor)
    loss_test = criterion(outputs_test, y_test_tensor.unsqueeze(1))

print(f"Test Loss: {loss_test.item():.4f}")


Epoch 1/50, Training Loss: 0.2402, Validation Loss: 0.1687
Epoch 2/50, Training Loss: 0.1679, Validation Loss: 0.1446
Epoch 3/50, Training Loss: 0.1442, Validation Loss: 0.1486
Epoch 4/50, Training Loss: 0.1486, Validation Loss: 0.1429
Epoch 5/50, Training Loss: 0.1430, Validation Loss: 0.1257
Epoch 6/50, Training Loss: 0.1256, Validation Loss: 0.1104
Epoch 7/50, Training Loss: 0.1101, Validation Loss: 0.1014
Epoch 8/50, Training Loss: 0.1011, Validation Loss: 0.0950
Epoch 9/50, Training Loss: 0.0946, Validation Loss: 0.0848
Epoch 10/50, Training Loss: 0.0844, Validation Loss: 0.0680
Epoch 11/50, Training Loss: 0.0677, Validation Loss: 0.0474
Epoch 12/50, Training Loss: 0.0472, Validation Loss: 0.0306
Epoch 13/50, Training Loss: 0.0305, Validation Loss: 0.0232
Epoch 14/50, Training Loss: 0.0232, Validation Loss: 0.0172
Epoch 15/50, Training Loss: 0.0173, Validation Loss: 0.0131
Epoch 16/50, Training Loss: 0.0132, Validation Loss: 0.0249
Epoch 17/50, Training Loss: 0.0252, Validation Lo